In [4]:
# Import Library
import numpy as np
import pandas as pd
import os
import sqlalchemy as db
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine
import pymysql
import xlrd
import decimal
from datetime import date, datetime

D = decimal.Decimal


ModuleNotFoundError: No module named 'mysql'

In [ ]:
data_path1 = 'C:\\Users\\fwidjojoputra\\Desktop\\data\\'
data_path2 = 'E:\\EASi\\5-Operations\\50-Management\\500-DataCentral\\3-Super User\\Development\\data_repository\\msp_data\\'

file_name1 = os.path.join(data_path1,'resources.csv') 
file_name2 = os.path.join(data_path1,'projects.csv') 
file_name3 = os.path.join(data_path1,'tasks.csv') 
file_name4 = os.path.join(data_path1,'assignments.csv') 
file_name5 = os.path.join(data_path1,'tasktime_phased.csv')
file_name6 = os.path.join(data_path2,'psoft_data.csv')
file_name7 = os.path.join(data_path2,'psoft_prj_data.csv')

In [ ]:
df1 = pd.read_csv(file_name1) # res
df2 = pd.read_csv(file_name2) # prj
df3 = pd.read_csv(file_name3) # task
df4 = pd.read_csv(file_name4) # ass
df5 = pd.read_csv(file_name5) # task_tp
df6 = pd.read_csv(file_name6) # psoft_data
df7 = pd.read_csv(file_name7) # psoft__prj_data

In [ ]:
#-------------------- Data prep RESOURCES --------------------#
res_df = df1.loc[:,['DataCentralID',
                    'ResourceId']]

prj_df = df2.loc[:,['ProjectId',
                    'SOWNb',
                    'ProjectActualFinishDate',
                    'year']]

tsk_df = df3.loc[:,['TaskId',
                    'ParentTaskId',
                    'ProjectId',
                    'TaskName']]

ass_df = df4.loc[:,['ProjectId',
                    'TaskId',
                    'AssignmentId',
                    'ResourceId']]

tsktp_df = df5.loc[:,['TaskId',
                      'ProjectId',
                      'TaskWork',
                      'TaskCost',
                      'TaskName']] 

psoft_df = df6.loc[:,['Customer',
                      'Category',
                      'Order No',
                      'Job Req #',
                      'ID',
                      'Name',
                      'Earn Code',
                      'Blended Rate',
                      'Job Code']]

psoft_prj_df = df7.loc[:,['Project',
                          'Psoft Project ID',
                          'EASi ID',
                          'Job Req',
                          'Bill Rate']]

res_df.rename({'DataCentralID':'easi_id',
               'ResourceId':'res_id'},axis=1, inplace=True)

prj_df.rename({'SOWNb':'sow_no',
               'ProjectId':'prj_id',
               'ProjectActualFinishDate':'end_date'},axis=1, inplace=True)

tsk_df.rename({'TaskId':'task_id',
               'ParentTaskId':'parent_id',
               'ProjectId':'prj_id',
               'TaskName':'task_name'},axis=1, inplace=True)

ass_df.rename({'ProjectId':'prj_id',
                'TaskId':'task_id',
                'AssignmentId':'ass_id',
                'ResourceId':'res_id'},axis=1, inplace=True)

tsktp_df.rename({'TaskId':'task_id',
                 'ProjectId':'prj_id',
                 'TaskWork':'task_work',
                 'TaskCost':'task_cost',
                 'TaskName':'task_name'},axis=1, inplace=True)

psoft_df.rename({'Customer':'client',
                 'Category':'cat',
                 'Order No':'ord_no',
                 'Job Req #':'job_req',
                 'ID':'easi_id',
                 'Name':'res_name',
                 'Earn Code':'earn_code',
                 'Blended Rate':'br',
                 'Job Code':'job_code'},axis=1, inplace=True)

psoft_prj_df.rename({'Project':'sow_no',
                     'Psoft Project ID':'psoft_prj_id',
                     'EASi ID':'easi_id',
                     'Job Req':'job_req',
                     'Bill Rate':'br'},axis=1, inplace=True)

In [ ]:
prj_tsk_df = tsk_df.merge(prj_df, on='prj_id', how='left')
prj_task_ass_df = prj_tsk_df.merge(ass_df, on='task_id', how='inner')
prj_task_ass_res_df = prj_task_ass_df.merge(res_df, on='res_id', how='left')

# group dataset by task_id
task_timephased_df = tsktp_df.groupby('task_id')['task_work','task_cost'].sum()
tsktp_df1 = task_timephased_df.reset_index()
tsktp_df1['br'] = tsktp_df1['task_cost']/tsktp_df1['task_work']

all_df = prj_task_ass_res_df.merge(tsktp_df1, on='task_id', how='left')

In [ ]:
# convert easi id to int
all_df = all_df.fillna(value='0')
test_df = all_df[all_df.easi_id != 'xxx']
test_df['easi_id'] = test_df['easi_id'].astype('int64')
test_df['br'] = test_df['br'].astype('float64')

In [ ]:
# remove id 0 and 1
test_df2 = test_df[test_df.easi_id != 0]
test_df3 = test_df2[test_df2.easi_id != 1]
test_df = test_df3.round(2)
# Collect required data for merging
init_df = test_df.loc[:,['sow_no',
                        'task_name',
                        'easi_id',
                        'br']]

In [ ]:
init_df['easi_id'].nunique(), init_df.shape
init_df['br'].nunique()

In [ ]:
# Handle psoft data
# psoft_df['easi_id'].nunique(), psoft_df.shape
psoft_df['br'].nunique()

In [ ]:
# Combined set
final_df = init_df.merge(psoft_df, left_on=['easi_id','br'], right_on=['easi_id','br'], how='inner')
final_df.shape

In [ ]:
final_df.shape, psoft_prj_df.shape

In [ ]:
extract_df = final_df.merge(psoft_prj_df, left_on=['sow_no','easi_id','br','job_req'], right_on=['sow_no','easi_id','br','job_req'], how='inner')
extract_df.head(50) 